In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
for dirname, _, filenames in os.walk('models'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

models\t2t_vit.py
models\t2t_vit_dense.py
models\t2t_vit_ghost.py
models\t2t_vit_se.py
models\token_performer.py
models\token_transformer.py
models\transformer_block.py
models\__init__.py
models\__pycache__\t2t_vit.cpython-311.pyc
models\__pycache__\t2t_vit_dense.cpython-311.pyc
models\__pycache__\t2t_vit_ghost.cpython-311.pyc
models\__pycache__\t2t_vit_se.cpython-311.pyc
models\__pycache__\token_performer.cpython-311.pyc
models\__pycache__\token_transformer.cpython-311.pyc
models\__pycache__\transformer_block.cpython-311.pyc
models\__pycache__\__init__.cpython-311.pyc


In [3]:
def load_state_dict(checkpoint_path, use_ema=False, num_classes=1000):
    if checkpoint_path and os.path.isfile(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        state_dict_key = 'state_dict'
        if isinstance(checkpoint, dict):
            if use_ema and 'state_dict_ema' in checkpoint:
                state_dict_key = 'state_dict_ema'
        if state_dict_key and state_dict_key in checkpoint:
            new_state_dict = OrderedDict()
            for k, v in checkpoint[state_dict_key].items():
                # strip `module.` prefix
                name = k[7:] if k.startswith('module') else k
                new_state_dict[name] = v
            state_dict = new_state_dict
        else:
            state_dict = checkpoint
#         _logger.info("Loaded {} from checkpoint '{}'".format(state_dict_key, checkpoint_path))
#         if num_classes != 1000:
#             # completely discard fully connected for all other differences between pretrained and created model
#             del state_dict['head' + '.weight']
#             del state_dict['head' + '.bias']

        return state_dict
    else:
#         _logger.error("No checkpoint found at '{}'".format(checkpoint_path))
        raise FileNotFoundError()


def load_for_transfer_learning(model, checkpoint_path, use_ema=False, strict=False, num_classes=1000):
    state_dict = load_state_dict(checkpoint_path, use_ema, num_classes)
    model.load_state_dict(state_dict, strict=strict)

In [6]:
import torch

In [8]:
stat = torch.load('80.7_T2T_ViT_t_14.pth.tar')

# check if state_dict_ema exists if so del
if 'state_dict_ema' in stat:
    del stat['state_dict_ema']['head.weight']
    del stat['state_dict_ema']['head.bias']

torch.save(stat['state_dict_ema'],'80.7_T2T_ViT_t_14.pth.tar')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20028\1164165959.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  stat = torch.load('80.7_T2T_ViT_t_14.pth.tar')


KeyError: 'state_dict_ema'

In [ ]:
from models.t2t_vit import *
# create model
model = T2t_vit_14(img_size=224, num_classes=12)
# load_for_transfer_learning(model, '80.7_T2T_ViT_t_14.pth.tar', num_classes=12)